## Исследование надёжности заёмщиков

## План работы:
[Шаг 1. Знакомство с данными](#step1)

[Шаг 2. Предобработка данных:](#step2)

- [обработка пропусков и дубликатов](#step2.1)
- [изменение типов данных](#step2.2)
- [лемматизация](#step2.3)
- [категоризация](#step2.4)

[Шаг 3. Анализ данныx](#step3)

## Шаг 1. Знакомство с данными <a id='step1'></a>

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
data = pd.read_csv('/datasets/data.csv') #чтение файла  с данными
data.info() #вывод общей информации о данных для изучения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.head(5) #знакомство со структурой таблицы и данными

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Итог первого этапа работы

Есть пропущенные значения в столбцах о стаже **`days_employed`** и ежемесячном доходе **`total_income`**. Стаж нам не интересен для анализа в данной задаче, про него забываем. 
Про доход - нам нужно решить удалять строки с пропусками или проставить какие-то средние показатели, разбив на категории по типу занятости, по возрасту, или возможно по уровню образования?

В колонке об образовании **`education`** - даже без программной порверки сразу видны дубликаты, отличающиеся регистром. Необходимо проверить и обработать программно.

Доход **`total_income`** - вещественный, в принципе для реальности ничего в этом ужасного нет, доход может быть указан с копейками, или в тыс руб с несколькими знаками после запятой. Для удобства и экономии ресурсов переведем в целочисленный тип, так как это не приведет к потерям при анализе этого показателя.

## Шаг 2. Предобработка данных <a id='step2'></a>

- обработка пропусков и дубликатов <a id='step2.1'></a>

Уровень дохода **`total_income`** важный показатель  в данной задаче, просто удаление строк с отсутствующими показателями уровня доходов не подойдет. Почему они там появились нам неизвестно, так как мы анализируем данные по кредитной истории, вряд ли кредит выдавался без указания дохода.
Нужно попытаться заполнить пропуски по данному показателю характерными значениями.
Для это проанализируем показатель, сгруппировав исходные данные, поробуем по разным категориям. Например, по уровню образования и типу занятости. Чтобы экстремальные значения показателя не повлияли на анализ и получились более объективные значения, конкретно для этого показателя будем использовать метод **median()**

Сгруппируем данные по уровню образования, предварительно избавившись от дубликатов в столбце **`education`** приведением к нижнему регистру, используя **str.lower()**. Можно было сразу группировать по **`education_id`**, там нет дубликатов, а результат был бы точно такой же, но с наименованием образования, а не с id таблицы смотрятся понятнее и удобнее.

In [4]:
data['education'] = data['education'].str.lower() #обработка пропусков

data.groupby('education')['total_income'].median() #группировка по уровню образования

education
высшее                 175340.818855
начальное              117137.352825
неоконченное высшее    160115.398644
среднее                136478.643244
ученая степень         157259.898555
Name: total_income, dtype: float64

Сгруппируем по типу занятости:

In [5]:
data.groupby('income_type')['total_income'].median() #группировка по типу занятости

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [6]:
#сводная таблица среднего дохода по уровню образования и типу занятости
data_pivot = data.pivot_table(index=['income_type'], columns='education', values='total_income', 
                              aggfunc='median')
data_pivot.head(10) 

education,высшее,начальное,неоконченное высшее,среднее,ученая степень
income_type,,,,,
безработный,202722.511368,NaN,NaN,59956.991984,NaN
в декрете,NaN,NaN,NaN,53829.130729,NaN
госслужащий,172511.107016,148339.290825,160592.345303,136652.970357,111392.231107
компаньон,201785.400018,136798.905143,179867.152890,159070.690289,NaN
пенсионер,144240.768611,102598.653164,120136.896353,114842.854099,177088.845999
предприниматель,499163.144947,NaN,NaN,NaN,NaN
сотрудник,165640.744634,125994.910603,151308.937846,136555.108821,198570.757322
студент,98201.625314,NaN,NaN,NaN,NaN


По моему мнению необходимо заполнять пропуски, используя средние значения, сгруппированные по типу занятости **`income_type`**, так как по сводной таблице видно, что у группы с одинаковым уровнем образования, значения дохода по типу занятости могут отличаться в два и более раз. Чтобы не присвоить, например, студенту или декретнице, несвойственно высокие значения, исходя из их уровня образования, произведем замену пропусков на характерные значения, исходя из типа занятости:

In [7]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))
#проверим все ли пропуски заполнились
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [8]:
# проверим, что замена произошла корректно, средние значение по типу занятости не поменялись
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Дубликаты в названиях уровня образования обработаны выше приведением всех значений столбца к нижнему регистру с помощью str.lower(). Проверим есть ли дубликаты строк с помощью метода **duplicated()**.

In [9]:
data.duplicated().sum()

71

Дубликатов очень мало относительно данных. И так как в таблице нет какого-нибудь ID клиента и даты кредита (так как теоретически один и тот же человек мог взять два кредита в разный период), мы не можем определить наверняка, истинные это дубликаты или просто совпадения у разных клиентов. Например, до замены типа данных столбца **`total_income`** из вещественных в целочисленный дубликатов было еще меньше, всего 54. 
В любом случае их настолько мало (меньше четверти процента), что на результаты исследования не повлияет удалим мы эти данные или оставим. Можно удалить методом **drop_duplicates()**, и делать это до замены типов.

- замена типа данных <a id='step2.2'></a>

В начале нашего исследования, чмы определили, что будем менять вещественный тип данных в столбцe **`total_income`** на целочисленный. Для этого будем использовать **astype()** с аргументом **int**

In [10]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [11]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


- лемматизация <a id='step2.3'></a>

Одна из задач проекта, проследить зависимость задолженности по кредиту от цели кредитования. Для ответа на этот вопрос лемматизируем столбец **`purpose`** - цель кредита

In [12]:
m = Mystem()
def lem(row):
    lemmas = ' '.join(m.lemmatize(row))
    return lemmas

data['lemmas'] = data['purpose'].apply(lem)

data.head()       

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n


In [13]:
join_purpose = ' '.join(data['purpose'])
join_lemmas = m.lemmatize(join_purpose)
print(Counter(join_lemmas))

Counter({' ': 55201, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'подержать': 858, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'приобретение': 462, 'профильный': 436, 'подержанный': 110, '\n': 1})


Таким образом определены категории: недвижимость, жилье, автомобиль, образование, свадьба. И слова "паразиты" вроде 'покупка', 'проведение' и т.д., которые можно игнорировать. 
Но тут единственное, что бросается в глаза, нужно уточнять значения некоторых слов, в каком контексте они используются: например "операция", если бы это слово было связано с лечением, то это была бы отдельная категория. Но в нашем случае это всегда были операции с недвижимостью, поэтому можно игнорировать.

- категоризация данных <a id='step2.4'></a>

Исходя из целей кредита выделим следующие категории:

In [14]:
category_purpose = ['свадьба','недвижимость','жилье','автомобиль','образование']

Присвоим соответствующую категорию:

In [15]:
def category(row):
    for element in category_purpose:
        if element in row:
            return element

data['category'] = data['lemmas'].apply(category)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба


## Шаг 3. Анализ данных <a id='step3'></a>

### Вопрос 1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посмотрим какие есть значения в столбце Дети:

In [16]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Предположим, что -1 и 20 это опечатки, случайно '-', '0' нажались а на самом деле это 1 и 2 ребенка, соответственно. Чтобы проверить, как наличие детей влияет на возврат кредита в срок, добавим столбец наличие детей, раз в вопросе принципиально наличие, а не количество, главное, что они есть, заодно и эти непонятные значения попадут в категорию 'есть дети' без дополнительных преобразований данных:

In [17]:
#это можно было сделать в разделе Категоризация
def yes_no_children(children):
        if children == 0:
                return 'нет детей'
        return 'есть дети'

data['yes_no_children'] = data['children'].apply(yes_no_children)
data['yes_no_children'].value_counts()


нет детей    14149
есть дети     7376
Name: yes_no_children, dtype: int64

In [18]:
# функция для свода данных, пригодится дальше  
def data_pivot_final(data, index):
    data_pivot = data.pivot_table(index=[index], columns='debt', values='total_income', 
                              aggfunc='count')
    data_pivot['total'] = data_pivot[0] + data_pivot[1]
    data_pivot['proportion_debt'] = data_pivot[1]/data_pivot['total']
    return data_pivot.sort_values(by = 'proportion_debt', ascending = False)

data_pivot_final(data, 'yes_no_children')



debt,0,1,total,proportion_debt
yes_no_children,,,,
есть дети,6698,678,7376,0.091920
нет детей,13086,1063,14149,0.075129


### Ответ 1:

Из 7376 клиентов с детьми: 678 - не вернули кредит в срок, это примерно 9.2%.
Из 14149 клиентов, не имеющих детей: 1063 - не вернули кредит в срок, это 7.5%
Всего доля, имеющих задолженность, независимо от количества детей составила - 8.1%. 
Клиенты с детьми менее добросовестные, чем клиенты без детей. Хотя до начала анализа, я бы предположила, что клиенты с детьми более ответственные люди и обязательства перед банком погашают в срок.

### Вопрос 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посмотрим какие значения принимаются в столбце семейный статус:

In [19]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Дубликатов нет, можно оставить категории семейного статуса как есть.

In [20]:
data_pivot_final(data, 'family_status')

debt,0,1,total,proportion_debt
family_status,,,,
Не женат / не замужем,2539,274,2813,0.097405
гражданский брак,3789,388,4177,0.092890
женат / замужем,11449,931,12380,0.075202
в разводе,1110,85,1195,0.071130
вдовец / вдова,897,63,960,0.065625


### Ответ 2:

Наибольшая доля недобросовестных плательщиков среди клиентов со статусами "не женат/не замужем" и "гражданский брак" - более 9% имели задолженность по возврату кредитов. Наименьшая доля среди должников у клиентов с семейным статусом "вдовец/вдова" - 6.5%. А в категориях женат/замужем и в разводе должников чуть больше 7%. Но и здесь разброс небольшой, явных аутсайдеров нет, кому можно было бы сразу отказывать, например, в выдаче кредита.

### Вопрос 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Посмотрим какие значения есть в столбце Доход:

In [21]:
data['total_income'].min()

20667

In [22]:
data['total_income'].max()

2265604

In [23]:
data['total_income'].median()

142594.0

**Разброс в доходах существенный. Опираясь на экстремумы и средние, попробуем поделить доход на следующие категории:**
- менее 150 тыс - низкий (всё, что ниже медианы (округлила))
- 150 - 300 тыс - средний (от медианы и в два раза выше, хотя может есть какой-то более научный подход, но опираться не на что, исходных данных, какая это страна, или город, какой средний доход населения - ничего нет)
- более 300 тыс - высокий

In [24]:
def total_income_group(total_income):
    if total_income <= 150000:
        return 'низкий'
    if total_income <= 300000:
        return 'средний'
    return 'высокий'

data['total_income_group'] = data['total_income'].apply(total_income_group)
data['total_income_group'].value_counts()

низкий     11685
средний     8357
высокий     1483
Name: total_income_group, dtype: int64

Группы получились неравномерные, но на мой взгляд это логично. Люди с высоким доходом если и берут кредит, то на крупные сделки с недвижимостью - таких по статистике должно быть меньше, чем кредитов, например, на автомоби.

Сведем данные в таблицу:

In [25]:
data_pivot_final(data, 'total_income_group')

debt,0,1,total,proportion_debt
total_income_group,,,,
низкий,10707,978,11685,0.083697
средний,7700,657,8357,0.078617
высокий,1377,106,1483,0.071477


### Ответ 3:

Наибольшая доля просрочников среди клиентов с низким уровнем дохода, что и следовало ожидать. Самые добросовестные плательщики - с высоким уровнем дохода.

### Вопрос 4. Как разные цели кредита влияют на его возврат в срок?

Посмотрим наши категории целей кредита:

In [26]:
data['category'].value_counts()

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: category, dtype: int64

Сведем в таблицу:

In [27]:
data_pivot_final(data, 'category')

debt,0,1,total,proportion_debt
category,,,,
автомобиль,3912,403,4315,0.093395
образование,3652,370,4022,0.091994
свадьба,2162,186,2348,0.079216
недвижимость,5893,474,6367,0.074446
жилье,4165,308,4473,0.068858


### Ответ 4:

Кредиты на покупку автомобиля и на образование чаще всех не возвращались вовремя - доля недобросовестных клиентов более 9%. Далее идут: свадьба - почти 8%, недвижимость - 7.4% и жилье 6.9%. Категории недвижимость и жилье возможно было бы объединить в одну категорию, но на выводы в целом это не повлияло бы, они остались бы на первом месте по качеству возвратов кредитов в срок.

## Итог проекта

Мы прошли все этапы анализа и обработки данных, ответили на все поставленные вопросы. На мой взгляд, зависимости хоть прослеживаются, но не ярко выражены. Наверное, нужно анализировать в совокупности. Например, посчитать какова вероятность возврата в срок кредита на покупку автомобиля у неженатого пенсионера с пятью детьми с низким уровнем дохода. Или: какие риски возникнут у кредитной организации, если ты вдовец без детей с высоким уровнем дохода хочешь взять кредит на образование.